In [1]:
# https://unofficed.com/courses/mastering-algotrading-a-beginners-guide-using-kiteconnect-api/lessons/organizing-expiry-dates-of-financial-instruments-with-python/

In [2]:
import logging
from kiteconnect import KiteConnect

logging.basicConfig(level=logging.DEBUG)

kite = KiteConnect(api_key="your_api_key")

# Redirect the user to the login url obtained
# from kite.login_url(), and receive the request_token
# from the registered redirect url after the login flow.
# Once you have the request_token, obtain the access_token
# as follows.

data = kite.generate_session("request_token_here", api_secret="your_secret")
kite.set_access_token(data["access_token"])

/tmp/ipykernel_4126949/1105643221.py:8: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  from kite.kitecustom import *


In [3]:
def create_expiry_list(df):
    expiry_list = {"fut": {}, "options": {}}

    # Filtering for Futures
    df_fut = df[df['instrument_type'] == 'FUT']
    unique_names_fut = df_fut['name'].unique()
    for name in unique_names_fut:
        expiry_dates = df_fut[df_fut['name'] == name]['expiry'].dropna().unique()
        expiry_dates = pd.to_datetime(expiry_dates)
        expiry_list["fut"][name] = sorted(expiry_dates)

    # Filtering for Options (CE and PE)
    df_options = df[df['instrument_type'].isin(['CE', 'PE'])]
    unique_names_options = df_options['name'].unique()
    for name in unique_names_options:
        expiry_dates = df_options[df_options['name'] == name]['expiry'].dropna().unique()
        expiry_dates = pd.to_datetime(expiry_dates)
        expiry_list["options"][name] = sorted(expiry_dates)

    # Converting numpy datetime64 to string in 'YYYY-MM-DD' format
    for instrument_type in expiry_list:
        for name in expiry_list[instrument_type]:
            expiry_list[instrument_type][name] = [date.strftime('%Y-%m-%d') for date in expiry_list[instrument_type][name]]

    return json.dumps(expiry_list, indent=4)

stockwise_json = json.loads(create_expiry_list(instrumentList))
stockwise_json

{'fut': {'EURINR': ['2023-11-24',
   '2023-11-28',
   '2023-12-01',
   '2023-12-08',
   '2023-12-15',
   '2023-12-22',
   '2023-12-27',
   '2023-12-29',
   '2024-01-05',
   '2024-01-12',
   '2024-01-19',
   '2024-01-25',
   '2024-01-29',
   '2024-02-02',
   '2024-02-09',
   '2024-02-27',
   '2024-03-27',
   '2024-04-26',
   '2024-05-29',
   '2024-06-26',
   '2024-07-29',
   '2024-08-28',
   '2024-09-26',
   '2024-10-29'],
  'GBPINR': ['2023-11-24',
   '2023-11-28',
   '2023-12-01',
   '2023-12-08',
   '2023-12-15',
   '2023-12-22',
   '2023-12-27',
   '2023-12-29',
   '2024-01-05',
   '2024-01-12',
   '2024-01-19',
   '2024-01-25',
   '2024-01-29',
   '2024-02-02',
   '2024-02-09',
   '2024-02-27',
   '2024-03-27',
   '2024-04-26',
   '2024-05-29',
   '2024-06-26',
   '2024-07-29',
   '2024-08-28',
   '2024-09-26',
   '2024-10-29'],
  'JPYINR': ['2023-11-24',
   '2023-11-28',
   '2023-12-01',
   '2023-12-08',
   '2023-12-15',
   '2023-12-22',
   '2023-12-27',
   '2023-12-29',
   '2024-

In [4]:
def create_expiry_list2(df):
    expiry_list = {}

    # Converting expiry column to datetime and dropping NaT
    df = df.dropna(subset=['expiry'])
    df['expiry'] = pd.to_datetime(df['expiry'])

    # Getting unique expiry dates and sorting them
    unique_expiry_dates = sorted(df['expiry'].unique())

    for date in unique_expiry_dates:
        # Converting numpy datetime64 to pandas Timestamp
        date = pd.Timestamp(date)
        date_str = date.strftime('%Y-%m-%d')
        expiry_list[date_str] = {"fut": [], "options": []}

        # Filtering DataFrame for each unique expiry date
        df_date = df[df['expiry'] == date]

        # Adding futures symbols
        fut_symbols = df_date[df_date['instrument_type'] == 'FUT']['name'].unique()
        expiry_list[date_str]['fut'] = sorted(fut_symbols)

        # Adding options symbols
        option_symbols = df_date[df_date['instrument_type'].isin(['CE', 'PE'])]['name'].unique()
        expiry_list[date_str]['options'] = sorted(option_symbols)

    return json.dumps(expiry_list, indent=4)

datewise_json = json.loads(create_expiry_list2(instrumentList))
datewise_json

/tmp/ipykernel_4126949/3727103880.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['expiry'] = pd.to_datetime(df['expiry'])


{'2023-11-23': {'fut': [], 'options': ['NATURALGAS']},
 '2023-11-24': {'fut': ['BANKEX',
   'EURINR',
   'GBPINR',
   'JPYINR',
   'MCXBULLDEX',
   'MIDCPNIFTY',
   'SENSEX',
   'USDINR'],
  'options': ['BANKEX',
   'EURINR',
   'GBPINR',
   'GOLD',
   'GOLDM',
   'JPYINR',
   'MIDCPNIFTY',
   'SENSEX',
   'SILVER',
   'USDINR']},
 '2023-11-27': {'fut': ['NATGASMINI', 'NATURALGAS'], 'options': []},
 '2023-11-28': {'fut': ['EURINR', 'FINNIFTY', 'GBPINR', 'JPYINR', 'USDINR'],
  'options': ['EURINR', 'FINNIFTY', 'GBPINR', 'JPYINR', 'USDINR']},
 '2023-11-29': {'fut': ['91DTB'], 'options': []},
 '2023-11-30': {'fut': ['654GS2032',
   '664GS2035',
   '667GS2035',
   '718GS2033',
   '718GS2037',
   '726GS2032',
   '726GS2033',
   '741GS2036',
   '754GS2036',
   'AARTIIND',
   'ABB',
   'ABBOTINDIA',
   'ABCAPITAL',
   'ABFRL',
   'ACC',
   'ADANIENT',
   'ADANIPORTS',
   'ALKEM',
   'ALUMINI',
   'ALUMINIUM',
   'AMBUJACEM',
   'APOLLOHOSP',
   'APOLLOTYRE',
   'ASHOKLEY',
   'ASIANPAINT',
  

In [5]:
def create_combined_expiry_list(df):
    # Getting the JSON data from both functions
    stockwise_json = json.loads(create_expiry_list(df))
    datewise_json = json.loads(create_expiry_list2(df))

    # Combining the JSON data
    combined_json = {
        "stockwise": stockwise_json,
        "datewise": datewise_json
    }

    # File path
    file_path = 'expiry.json'

    # Reading existing JSON data if file exists
    if os.path.exists(file_path):
        with open(file_path, 'r') as file:
            existing_json = json.load(file)

        # Merging the JSON data
        for leg in combined_json:
            if leg in existing_json:
                existing_json[leg].update(combined_json[leg])
            else:
                existing_json[leg] = combined_json[leg]
    else:
        existing_json = combined_json

    # Saving the merged JSON data
    with open(file_path, 'w') as file:
        json.dump(existing_json, file, indent=4)

    return json.dumps(existing_json, indent=4)


json.loads(create_combined_expiry_list(instrumentList))

/tmp/ipykernel_4126949/3727103880.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['expiry'] = pd.to_datetime(df['expiry'])


{'stockwise': {'fut': {'EURINR': ['2023-11-24',
    '2023-11-28',
    '2023-12-01',
    '2023-12-08',
    '2023-12-15',
    '2023-12-22',
    '2023-12-27',
    '2023-12-29',
    '2024-01-05',
    '2024-01-12',
    '2024-01-19',
    '2024-01-25',
    '2024-01-29',
    '2024-02-02',
    '2024-02-09',
    '2024-02-27',
    '2024-03-27',
    '2024-04-26',
    '2024-05-29',
    '2024-06-26',
    '2024-07-29',
    '2024-08-28',
    '2024-09-26',
    '2024-10-29'],
   'GBPINR': ['2023-11-24',
    '2023-11-28',
    '2023-12-01',
    '2023-12-08',
    '2023-12-15',
    '2023-12-22',
    '2023-12-27',
    '2023-12-29',
    '2024-01-05',
    '2024-01-12',
    '2024-01-19',
    '2024-01-25',
    '2024-01-29',
    '2024-02-02',
    '2024-02-09',
    '2024-02-27',
    '2024-03-27',
    '2024-04-26',
    '2024-05-29',
    '2024-06-26',
    '2024-07-29',
    '2024-08-28',
    '2024-09-26',
    '2024-10-29'],
   'JPYINR': ['2023-11-24',
    '2023-11-28',
    '2023-12-01',
    '2023-12-08',
    '2023-12